# Author collaboration clustering

##### The code below will allow to cluster based on authors collaboration, it uses as an input the file "paper_authors.csv" with two main columns "paper_id" and "author_id". Then a graph is created with this relations having the nodes as authors and the links as collaboration. And finally, the MCL clustering algorithm was used to cluster the graph.

### 1. Define MCL clustering algorithm

In [1]:
import sys
import numpy as np
import time
from optparse import OptionParser
import logging

def normalize(A):
    column_sums = A.sum(axis=0)
    new_matrix = A / column_sums[np.newaxis, :]
    return new_matrix

def inflate(A, inflate_factor):
    return normalize(np.power(A, inflate_factor))

def expand(A, expand_factor):
    return np.linalg.matrix_power(A, expand_factor)

def add_diag(A, mult_factor):
    return A + mult_factor * np.identity(A.shape[0])

def get_clusters(A):
    clusters = []
    for i, r in enumerate((A>0).tolist()):
        if r[i]:
            clusters.append(A[i,:]>0)

    clust_map  ={}
    for cn , c in enumerate(clusters):
        for x in  [ i for i, x in enumerate(c) if x ]:
            clust_map[cn] = clust_map.get(cn, [])  + [x]
    return clust_map

def draw(G, A, cluster_map):
    import networkx as nx
    import matplotlib.pyplot as plt

    clust_map = {}
    for k, vals in cluster_map.items():
        for v in vals:
            clust_map[v] = k

    colors = []
    for i in range(len(G.nodes())):
        colors.append(clust_map.get(i, 100))

    pos = nx.spring_layout(G)

    from matplotlib.pylab import matshow, show, cm
    plt.figure(2)
    nx.draw_networkx_nodes(G, pos,node_size = 200, node_color =colors , cmap=plt.cm.Blues )
    nx.draw_networkx_edges(G,pos, alpha=0.5)
    matshow(A, fignum=1, cmap=cm.gray)
    plt.show()
    show()


def stop(M, i):

    if i%5==4:
        m = np.max( M**2 - M) - np.min( M**2 - M)
        if m==0:
            logging.info("Stop at iteration %s" % i)
            return True

    return False


def mcl(M, expand_factor = 2, inflate_factor = 2, max_loop = 10 , mult_factor = 1):
    M = add_diag(M, mult_factor)
    M = normalize(M)

    for i in range(max_loop):
        logging.info("loop", i)
        M = inflate(M, inflate_factor)
        M = expand(M, expand_factor)
        if stop(M, i): break

    clusters = get_clusters(M)
    return M, clusters

def networkx_mcl(G, expand_factor = 2, inflate_factor = 2, max_loop = 10 , mult_factor = 1):
    import networkx as nx
    A = nx.adjacency_matrix(G)
    return mcl(np.array(A.todense()), expand_factor, inflate_factor, max_loop, mult_factor)

def print_info(options):
    print("-"*60)
    print("MARKOV CLUSTERING:")
    print("-" * 60)
    print("  expand_factor: %s" % options.expand_factor)
    print("  inflate_factor: %s" % options.inflate_factor)
    print("  mult factor: %s" % options.mult_factor)
    print("  max loops: %s\n" % options.max_loop)

def get_options():
    usage = "usage: %prog [options] <input_matrix>"
    parser = OptionParser(usage)
    parser.add_option("-e", "--expand_factor",
                      dest="expand_factor",
                      default=2,
                      type=int,
                      help="expand factor (default: %default)")
    parser.add_option("-i", "--inflate_factor",
                      dest="inflate_factor",
                      default=2,
                      type=float,
                      help="inflate factor (default: %default)")
    parser.add_option("-m", "--mult_factor",
                      dest="mult_factor",
                      default=2,
                      type=float,
                      help="multiply factor (default: %default)")
    parser.add_option("-l", "--max_loops",
                      dest="max_loop",
                      default=60,
                      type=int,
                      help="max loops (default: %default)")
    parser.add_option("-o", "--output", metavar="FILE", 
                      help="output (default: stdout)")

    parser.add_option("-v", "--verbose",
                      action="store_true", dest="verbose", default=True,
                      help="verbose (default: %default)")
    parser.add_option("-d", "--draw-graph",
                      action="store_true", dest="draw", default=False,
                      help="show graph with networkx (default: %default)")
    

    (options, args) = parser.parse_args()

    try:
        filename = args[0]
    except:
        raise Exception('input', 'missing input filename')


    return options, filename

def get_graph(csv_filename):
    import networkx as nx

    M = []
    for r in open(csv_filename):
        r = r.strip().split(",")
        M.append( map( lambda x: float(x.strip()), r))

    G = nx.from_numpy_matrix(np.matrix(M))
    return np.array(M), G

def clusters_to_output(clusters, options):
    if options.output and len(options.output)>0:
        f = open(options.output, 'w')
        for k, v in clusters.items():
            f.write("%s|%s\n" % (k, ", ".join(map(str, v)) ))
        f.close()
    else:    
        print("Clusters:")
        for k, v in clusters.items():
            print(k, v)

if __name__ == '__main__':

    options, filename = get_options()
    print_info(options)
    M, G = get_graph(filename)

    print(" number of nodes: %s\n" % M.shape[0])

    print(time.time(), "evaluating clusters...")
    M, clusters = networkx_mcl(G, expand_factor = options.expand_factor,
                               inflate_factor = options.inflate_factor,
                               max_loop = options.max_loop,
                               mult_factor = options.mult_factor)
    print(time.time(), "done\n")

    clusters_to_output(clusters, options)

    if options.draw:
        print(time.time(), "drawing...")
        draw(G, M, clusters)
        print(time.time(), "done")
#https://github.com/koteth/python_mcl

Usage: ipykernel_launcher.py [options] <input_matrix>

ipykernel_launcher.py: error: no such option: -f


SystemExit: 2

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2889: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


### 2. Read the file "paper_authors.csv" and transform the data into a two dimensions list with the pairs of all possible authors collaborating each other

In [2]:
import networkx as nx
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from pathlib import Path
d = Path().resolve().parent.parent
d = str(d) + "/data/original/paper_authors.csv"

csv = pd.read_csv(d)
#csv = pd.read_csv('paper_authors.csv')
paper_id = {}
for it in range(0,len(csv)):
    num = csv['paper_id'][it]
    if num in paper_id:
        paper_id[num].append(csv['author_id'][it])
    else:
        paper_id[num] = []
        paper_id[num].append(csv['author_id'][it])
lista = []
for it in paper_id.items():
    if(len(it[1])>=2):
        au_paper = 0
        for au in range(0,len(it[1])-1):
            for au2 in range(au+1,len(it[1])):
                x = [it[1][au],it[1][au2]]
                lista.append(x)

print("Number of rows: ",len(lista))

Number of rows:  21817


#### Remove duplicates in collaboration, so its ensure to have unique author pairs

In [3]:
#Remove duplicates
newListaF = sorted(lista)
listaFinal = [newListaF[i] for i in range(len(newListaF)) if i==0 or newListaF[i] != newListaF[i-1]]

#### Create a list of unique authors

In [4]:
### 1. Get the list of unique nodes in the graph
flist=[]
for x in listaFinal:
    for y in range(2):
        if x[y] not in flist:
            flist.append(x[y])

### 3. Create graph of authors with library "networkx"

In [5]:
#Graph creation
G=nx.Graph(name="Authors graph")
G.add_edges_from(listaFinal)

print("Number of nodes: ",G.number_of_nodes())
print("Number of edges: ",G.number_of_edges())

Number of nodes:  8457
Number of edges:  18451


### 4. Create clusters using the algorithm MCL clustering, the input is the graph from networkx

In [6]:
#As smaller the inflation as bigger the size of the clusters
M, clusters = networkx_mcl(G,inflate_factor=1.7)

#### The relation between author and cluster is made

In [7]:
## 2. Assign the node to each cluster
fclusters={}
for x in clusters:
    #print(clusters[x])
    fclusters[flist[x]] = clusters[x]

#### Clusters are being name, with format "Cluster[No.]"

In [8]:
## 3. Assign to each node the cluster name in format "Cluster[No.]"
d = fclusters
set_dict = list(enumerate(set(tuple(i) for i in d.values()), 1)) 
final_dict = { k: 'Cluster' + str([i for i,j in set_dict if list(j) == v][0]) for k,v in d.items() }

### 5. Create a CSV file with the final result, a table with columns "author_id" and "cluster_id"

In [10]:
#Save the dictionary as CSV
import csv
w = csv.writer(open("output_support.csv", "w"))
for key, val in final_dict.items():
    w.writerow([key, val])
with open('output_support.csv') as input, open('author-collaboration-clustering.csv', 'w') as output:
    non_blank = (line for line in input if line.strip())
    output.writelines(non_blank)

### 6. Function to evaluate the graph and the similarity among authors, with SimRank

In [11]:
#Chose relevant inflate factor
#Compute the SimRank between nodes from the same cluster

In [20]:
from collections import defaultdict
import copy
import sys

def simrank(G, r=0.9, max_iter=100):
    # init. vars
    sim_old = defaultdict(list)
    sim = defaultdict(list)
    for n in G.nodes():
        sim[n] = defaultdict(int)
        sim[n][n] = 1
        sim_old[n] = defaultdict(int)
        sim_old[n][n] = 0

    # recursively calculate simrank
    for iter_ctr in range(max_iter):
        if _is_converge(sim, sim_old):
            break
        sim_old = copy.deepcopy(sim)
        for u in G.nodes():
            for v in G.nodes():
                if u == v:
                    continue
                s_uv = 0.0
                for n_u in G.neighbors(u):
                    for n_v in G.neighbors(v):
                        s_uv += sim_old[n_u][n_v]
                sim[u][v] = (r * s_uv / (len(G.neighbors(u)) * len(G.neighbors(v))))
    return sim

def _is_converge(s1, s2, eps=1e-4):
    for i in s1.keys():
        for j in s1[i].keys():
            if abs(s1[i][j] - s2[i][j]) >= eps:
                return False
    return True

In [23]:
#simrank(G)

#### Example of how simrank works and the output

In [22]:
#import networkx
#import networkx_addon
G1 = nx.Graph()
G1.add_edges_from([('a','b'), ('b','c'), ('a','c'), ('c','d')])
simrank(G1)
#s = networkx_addon.similarity.simrank(G1)

defaultdict(list,
            {'a': defaultdict(int,
                         {'a': 1,
                          'b': 0.653792211016936,
                          'c': 0.6260762680740787,
                          'd': 0.7317028881451203}),
             'b': defaultdict(int,
                         {'a': 0.653792211016936,
                          'b': 1,
                          'c': 0.6260762680740787,
                          'd': 0.7317028881451203}),
             'c': defaultdict(int,
                         {'a': 0.6260762680740789,
                          'b': 0.6260762680740789,
                          'c': 1,
                          'd': 0.5365354388877558}),
             'd': defaultdict(int,
                         {'a': 0.7317028881451202,
                          'b': 0.7317028881451202,
                          'c': 0.5365354388877557,
                          'd': 1})})